#### Computing du score 1 à partir du TMJA national

##### PARTIE 1 : CONSTRUCTION DU GRAPH ET CALCUL DU BETWEENESS

Notebook amorcé par : Jesshuan

Objectifs :

- Récupération et ré-arrangement des fonctions du notebook de computation du score 1 dans l'objectif de générer des points routiers du réseau national (tmja 2018) dans Neo4j

Imports :

In [1]:
import geopandas as gpd
import pandas as pd
import geopy.distance
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import copy

from neo4j import GraphDatabase, basic_auth

In [2]:
# data_path = 'C:/Users/demo/Desktop/Lattitude/datas/'
data_path = 'datas'
os.makedirs(data_path, exist_ok=True)

In [40]:
# OPTIONNAL : Import et test avec TMJA 2019, qui est un fichier plus incomplet (et routes encore plus discontinues...) que 2018

'''# tmja = gpd.read_file('C:/Users/demo/Desktop/Lattitude/datas/tmja2018-shp/TMJA2018.shp')
file_name = 'TMJA2019'
ext = '.shp'
url = 'https://static.data.gouv.fr/resources/trafic-moyen-journalier-annuel-sur-le-reseau-routier-national/20211222-170254/tmja2019-shp.zip'
temp_path = 'temp_unzip'

try: 
    tmja = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    # From url
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    tmja = gpd.read_file(os.path.join(temp_path,file_name + ext))
    shutil.rmtree(temp_path)

    # From local file
    # tmja = gpd.read_file(os.path.join(data_path,file_name + ext))

    # avoiding multi dowloads for GES emmision lowering
    tmja.to_feather(os.path.join(data_path,file_name + '.feather'))'''

"# tmja = gpd.read_file('C:/Users/demo/Desktop/Lattitude/datas/tmja2018-shp/TMJA2018.shp')\nfile_name = 'TMJA2019'\next = '.shp'\nurl = 'https://static.data.gouv.fr/resources/trafic-moyen-journalier-annuel-sur-le-reseau-routier-national/20211222-170254/tmja2019-shp.zip'\ntemp_path = 'temp_unzip'\n\ntry: \n    tmja = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))\nexcept:\n    # From url\n    zip_file = requests.get(url).content\n    os.makedirs(temp_path, exist_ok=True)\n    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:\n        archive.extractall(temp_path)\n    tmja = gpd.read_file(os.path.join(temp_path,file_name + ext))\n    shutil.rmtree(temp_path)\n\n    # From local file\n    # tmja = gpd.read_file(os.path.join(data_path,file_name + ext))\n\n    # avoiding multi dowloads for GES emmision lowering\n    tmja.to_feather(os.path.join(data_path,file_name + '.feather'))"

In [3]:
# Load the data for French garge points
#file_name = 'dataset_charge_points.feather'

#bornes = gpd.read_feather(os.path.join(data_path, file_name))

# Load the data for the communes
#file_name = 'dataset_communes.feather'

#datas = gpd.read_feather(os.path.join(data_path, file_name))

file_name = 'TMJA2018'

tmja = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))

--------

Conversion dans le système ws84 (coordonnées GPS standard) :

In [4]:
tmja_wrs84 = tmja.to_crs(4326)

Remplissage par " capilarité" des missings values du tmja :

In [5]:
def split_and_fill_Local_null_values_by_key(df, column_key, missing_value=0,nb_rep=10, buffer_size=0.01):

    geometry_conservation = df.geometry

    df_clean = copy.copy(df)

    try:
        df_clean.reset_index(names="road_index", inplace=True)
    except Exception as e:
        print(e)
        pass

    df_buf = df_clean.buffer(buffer_size)

    df_clean.geometry = df_buf

    total_values = 0

    for k in range(nb_rep):

        new_column = []
        values = 0

        column_list = df_clean[column_key].to_list()
        index_list = df_clean['road_index'].to_list()

        df_join_road = df_clean.sjoin(df_clean, how='inner')
        df_join_road_clean = df_join_road.loc[df_join_road['road_index_left']!=df_join_road['road_index_right'],:]
        dict_new = df_join_road_clean[['road_index_left',f'{column_key}_right']].groupby('road_index_left').mean().to_dict()[f'{column_key}_right']

        for i, index in enumerate(index_list):
            old_tmja = column_list[i]
            if old_tmja == missing_value or str(old_tmja)==missing_value:
                try:
                    new_column.append(dict_new[index])
                    values+=1
                except:
                    new_column.append(old_tmja)
            else:
                new_column.append(old_tmja)
        
        df_clean[column_key] = new_column
    
        total_values += values

        print(f'... {k+1} rounds done. {values} are been modified... Total values modified : {total_values}')
    
    df_clean.geometry = geometry_conservation

    return df_clean

In [6]:
len(tmja_wrs84.loc[tmja_wrs84['tmja']==0,:])

1544

In [7]:
tmja_wrs84_clean = split_and_fill_Local_null_values_by_key(tmja_wrs84, column_key='tmja', nb_rep=40, buffer_size=0.003)

/tmp/ipykernel_11170/3422367474.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_buf = df_clean.buffer(buffer_size)


... 1 rounds done. 1539 are been modified... Total values modified : 1539
... 2 rounds done. 729 are been modified... Total values modified : 2268
... 3 rounds done. 459 are been modified... Total values modified : 2727
... 4 rounds done. 329 are been modified... Total values modified : 3056
... 5 rounds done. 227 are been modified... Total values modified : 3283
... 6 rounds done. 172 are been modified... Total values modified : 3455
... 7 rounds done. 123 are been modified... Total values modified : 3578
... 8 rounds done. 85 are been modified... Total values modified : 3663
... 9 rounds done. 61 are been modified... Total values modified : 3724
... 10 rounds done. 46 are been modified... Total values modified : 3770
... 11 rounds done. 42 are been modified... Total values modified : 3812
... 12 rounds done. 37 are been modified... Total values modified : 3849
... 13 rounds done. 35 are been modified... Total values modified : 3884
... 14 rounds done. 31 are been modified... Total va

-----

On récupère aussi les centres de routes :

In [9]:
tmja_wrs84_clean['Road_center'] = tmja_wrs84_clean.to_crs('+proj=cea').geometry.centroid.to_crs(tmja_wrs84.crs)

---

Réduction à nos besoins :

In [10]:
tmja_wrs84_clean.keys()

Index(['road_index', 'dateRefere', 'route', 'longueur', 'prD', 'depPrD',
       'concession', 'absD', 'cumulD', 'xD', 'yD', 'zD', 'prF', 'depPrF',
       'concessi_1', 'absF', 'cumulF', 'xF', 'yF', 'zF', 'cote', 'anneeMesur',
       'typeCompta', 'tmja', 'pctPL', 'geometry', 'Road_center'],
      dtype='object')

In [63]:
tmja_wrs84_reduced = tmja_wrs84_clean[['road_index','route','longueur','xD','yD','xF','yF','tmja','geometry','Road_center']]

In [64]:
tmja_wrs84_reduced['coords_D'] = gpd.points_from_xy(tmja_wrs84_reduced.xD, tmja_wrs84_reduced.yD, crs="EPSG:2154").to_crs(4326)

/home/jesshuan/miniconda3/envs/ECartoMobile/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [65]:
tmja_wrs84_reduced['coords_F'] = gpd.points_from_xy(tmja_wrs84_reduced.xF, tmja_wrs84_reduced.yF, crs="EPSG:2154").to_crs(4326)

/home/jesshuan/miniconda3/envs/ECartoMobile/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [66]:
tmja_wrs84_reduced

,road_index,route,longueur,xD,yD,xF,yF,tmja,geometry,Road_center,coords_D,coords_F
0,0,69D0301,6055.0,844036.84,6510806.45,849346.89,6509342.34,148668.000000,"LINESTRING (4.85057 45.68163, 4.85068 45.68180...",POINT (4.88473 45.68064),POINT (4.85057 45.68163),POINT (4.91831 45.66731)
1,1,69D0383,13752.0,846831.30,6522372.16,843537.87,6512429.80,92919.142857,"LINESTRING (4.89002 45.78515, 4.89044 45.78476...",POINT (4.88103 45.73453),POINT (4.89002 45.78515),POINT (4.84465 45.69635)
2,2,69D0383BPNL,2408.0,844982.46,6522772.27,846831.30,6522372.16,34071.666667,"LINESTRING (4.86635 45.78915, 4.86652 45.78922...",POINT (4.87845 45.79095),POINT (4.86635 45.78915),POINT (4.89002 45.78515)
3,3,75000C01,35040.0,655221.44,6858722.80,655221.44,6858722.80,75315.000000,"LINESTRING (2.28085 48.87830, 2.28107 48.87860...",POINT (2.33644 48.85780),POINT (2.39006 48.82701),POINT (2.39006 48.82701)
4,4,75000C02,35040.0,655227.28,6858703.75,655227.28,6858703.75,75315.000000,"LINESTRING (2.28063 48.87837, 2.28082 48.87862...",POINT (2.33640 48.85782),POINT (2.39014 48.82684),POINT (2.39014 48.82684)
...,...,...,...,...,...,...,...,...,...,...,...,...
5963,5963,N1547,2687.0,895516.86,6250912.48,892891.80,6250945.63,729.027778,"LINESTRING (5.40975 43.33051, 5.40962 43.33043...",POINT (5.39351 43.33022),POINT (5.40975 43.33051),POINT (5.37742 43.33153)
5964,5964,P0224,8995.0,555988.61,6288880.30,562812.02,6287175.90,7796.500000,"LINESTRING (1.21422 43.68430, 1.21448 43.68436...",POINT (1.25496 43.66936),POINT (1.21422 43.68430),POINT (1.29926 43.67032)
5965,5965,P0524,1777.0,440252.10,6361052.92,440343.36,6359374.43,1064.200000,"LINESTRING (-0.25658 44.30029, -0.25652 44.300...",POINT (-0.25450 44.29353),POINT (-0.25658 44.30029),POINT (-0.25457 44.28522)
5966,5966,P0524,3860.0,458798.85,6325912.32,458785.94,6322872.51,318.941667,"LINESTRING (-0.00751 43.99087, -0.00751 43.990...",POINT (-0.01313 43.97918),POINT (-0.00751 43.99087),POINT (-0.00623 43.96352)


In [67]:
tmja_wrs84_reduced.set_geometry('Road_center')

,road_index,route,longueur,xD,yD,xF,yF,tmja,geometry,Road_center,coords_D,coords_F
0,0,69D0301,6055.0,844036.84,6510806.45,849346.89,6509342.34,148668.000000,"LINESTRING (4.85057 45.68163, 4.85068 45.68180...",POINT (4.88473 45.68064),POINT (4.85057 45.68163),POINT (4.91831 45.66731)
1,1,69D0383,13752.0,846831.30,6522372.16,843537.87,6512429.80,92919.142857,"LINESTRING (4.89002 45.78515, 4.89044 45.78476...",POINT (4.88103 45.73453),POINT (4.89002 45.78515),POINT (4.84465 45.69635)
2,2,69D0383BPNL,2408.0,844982.46,6522772.27,846831.30,6522372.16,34071.666667,"LINESTRING (4.86635 45.78915, 4.86652 45.78922...",POINT (4.87845 45.79095),POINT (4.86635 45.78915),POINT (4.89002 45.78515)
3,3,75000C01,35040.0,655221.44,6858722.80,655221.44,6858722.80,75315.000000,"LINESTRING (2.28085 48.87830, 2.28107 48.87860...",POINT (2.33644 48.85780),POINT (2.39006 48.82701),POINT (2.39006 48.82701)
4,4,75000C02,35040.0,655227.28,6858703.75,655227.28,6858703.75,75315.000000,"LINESTRING (2.28063 48.87837, 2.28082 48.87862...",POINT (2.33640 48.85782),POINT (2.39014 48.82684),POINT (2.39014 48.82684)
...,...,...,...,...,...,...,...,...,...,...,...,...
5963,5963,N1547,2687.0,895516.86,6250912.48,892891.80,6250945.63,729.027778,"LINESTRING (5.40975 43.33051, 5.40962 43.33043...",POINT (5.39351 43.33022),POINT (5.40975 43.33051),POINT (5.37742 43.33153)
5964,5964,P0224,8995.0,555988.61,6288880.30,562812.02,6287175.90,7796.500000,"LINESTRING (1.21422 43.68430, 1.21448 43.68436...",POINT (1.25496 43.66936),POINT (1.21422 43.68430),POINT (1.29926 43.67032)
5965,5965,P0524,1777.0,440252.10,6361052.92,440343.36,6359374.43,1064.200000,"LINESTRING (-0.25658 44.30029, -0.25652 44.300...",POINT (-0.25450 44.29353),POINT (-0.25658 44.30029),POINT (-0.25457 44.28522)
5966,5966,P0524,3860.0,458798.85,6325912.32,458785.94,6322872.51,318.941667,"LINESTRING (-0.00751 43.99087, -0.00751 43.990...",POINT (-0.01313 43.97918),POINT (-0.00751 43.99087),POINT (-0.00623 43.96352)


In [68]:
tmja_wrs84_reduced['x_center'] = [np.round(elt.coords[0][0],5) for elt in tmja_wrs84_reduced['Road_center'].geometry]
tmja_wrs84_reduced['y_center'] = [np.round(elt.coords[0][1],5) for elt in tmja_wrs84_reduced['Road_center'].geometry]
tmja_wrs84_reduced['x_1'] = [np.round(elt.coords[0][0],5) for elt in tmja_wrs84_reduced['coords_D'].geometry]
tmja_wrs84_reduced['y_1'] = [np.round(elt.coords[0][1],5) for elt in tmja_wrs84_reduced['coords_D'].geometry]
tmja_wrs84_reduced['x_2'] = [np.round(elt.coords[0][0],5) for elt in tmja_wrs84_reduced['coords_F'].geometry]
tmja_wrs84_reduced['y_2'] = [np.round(elt.coords[0][1],5) for elt in tmja_wrs84_reduced['coords_F'].geometry]

/home/jesshuan/miniconda3/envs/ECartoMobile/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/jesshuan/miniconda3/envs/ECartoMobile/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/jesshuan/miniconda3/envs/ECartoMobile/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is tryin

In [69]:
tmja_wrs84_reduced

,road_index,route,longueur,xD,yD,xF,yF,tmja,geometry,Road_center,coords_D,coords_F,x_center,y_center,x_1,y_1,x_2,y_2
0,0,69D0301,6055.0,844036.84,6510806.45,849346.89,6509342.34,148668.000000,"LINESTRING (4.85057 45.68163, 4.85068 45.68180...",POINT (4.88473 45.68064),POINT (4.85057 45.68163),POINT (4.91831 45.66731),4.88473,45.68064,4.85057,45.68163,4.91831,45.66731
1,1,69D0383,13752.0,846831.30,6522372.16,843537.87,6512429.80,92919.142857,"LINESTRING (4.89002 45.78515, 4.89044 45.78476...",POINT (4.88103 45.73453),POINT (4.89002 45.78515),POINT (4.84465 45.69635),4.88103,45.73453,4.89002,45.78515,4.84465,45.69635
2,2,69D0383BPNL,2408.0,844982.46,6522772.27,846831.30,6522372.16,34071.666667,"LINESTRING (4.86635 45.78915, 4.86652 45.78922...",POINT (4.87845 45.79095),POINT (4.86635 45.78915),POINT (4.89002 45.78515),4.87845,45.79095,4.86635,45.78915,4.89002,45.78515
3,3,75000C01,35040.0,655221.44,6858722.80,655221.44,6858722.80,75315.000000,"LINESTRING (2.28085 48.87830, 2.28107 48.87860...",POINT (2.33644 48.85780),POINT (2.39006 48.82701),POINT (2.39006 48.82701),2.33644,48.85780,2.39006,48.82701,2.39006,48.82701
4,4,75000C02,35040.0,655227.28,6858703.75,655227.28,6858703.75,75315.000000,"LINESTRING (2.28063 48.87837, 2.28082 48.87862...",POINT (2.33640 48.85782),POINT (2.39014 48.82684),POINT (2.39014 48.82684),2.33640,48.85782,2.39014,48.82684,2.39014,48.82684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5963,5963,N1547,2687.0,895516.86,6250912.48,892891.80,6250945.63,729.027778,"LINESTRING (5.40975 43.33051, 5.40962 43.33043...",POINT (5.39351 43.33022),POINT (5.40975 43.33051),POINT (5.37742 43.33153),5.39351,43.33022,5.40975,43.33051,5.37742,43.33153
5964,5964,P0224,8995.0,555988.61,6288880.30,562812.02,6287175.90,7796.500000,"LINESTRING (1.21422 43.68430, 1.21448 43.68436...",POINT (1.25496 43.66936),POINT (1.21422 43.68430),POINT (1.29926 43.67032),1.25496,43.66936,1.21422,43.68430,1.29926,43.67032
5965,5965,P0524,1777.0,440252.10,6361052.92,440343.36,6359374.43,1064.200000,"LINESTRING (-0.25658 44.30029, -0.25652 44.300...",POINT (-0.25450 44.29353),POINT (-0.25658 44.30029),POINT (-0.25457 44.28522),-0.25450,44.29353,-0.25658,44.30029,-0.25457,44.28522
5966,5966,P0524,3860.0,458798.85,6325912.32,458785.94,6322872.51,318.941667,"LINESTRING (-0.00751 43.99087, -0.00751 43.990...",POINT (-0.01313 43.97918),POINT (-0.00751 43.99087),POINT (-0.00623 43.96352),-0.01313,43.97918,-0.00751,43.99087,-0.00623,43.96352


In [71]:
tmja_wrs84_reduced.keys()

Index(['road_index', 'route', 'longueur', 'tmja', 'x_center', 'y_center',
       'x_1', 'y_1', 'x_2', 'y_2'],
      dtype='object')

In [70]:
tmja_wrs84_reduced = tmja_wrs84_reduced[['road_index','route','longueur','tmja','x_center','y_center','x_1','y_1','x_2','y_2']]

In [72]:
tmja_wrs84_reduced

,road_index,route,longueur,tmja,x_center,y_center,x_1,y_1,x_2,y_2
0,0,69D0301,6055.0,148668.000000,4.88473,45.68064,4.85057,45.68163,4.91831,45.66731
1,1,69D0383,13752.0,92919.142857,4.88103,45.73453,4.89002,45.78515,4.84465,45.69635
2,2,69D0383BPNL,2408.0,34071.666667,4.87845,45.79095,4.86635,45.78915,4.89002,45.78515
3,3,75000C01,35040.0,75315.000000,2.33644,48.85780,2.39006,48.82701,2.39006,48.82701
4,4,75000C02,35040.0,75315.000000,2.33640,48.85782,2.39014,48.82684,2.39014,48.82684
...,...,...,...,...,...,...,...,...,...,...
5963,5963,N1547,2687.0,729.027778,5.39351,43.33022,5.40975,43.33051,5.37742,43.33153
5964,5964,P0224,8995.0,7796.500000,1.25496,43.66936,1.21422,43.68430,1.29926,43.67032
5965,5965,P0524,1777.0,1064.200000,-0.25450,44.29353,-0.25658,44.30029,-0.25457,44.28522
5966,5966,P0524,3860.0,318.941667,-0.01313,43.97918,-0.00751,43.99087,-0.00623,43.96352


--------------------------

Fonctions de requêtes neo4j :

In [73]:
def delete_all(tx):
    query = 'MATCH (n) \
                DETACH DELETE n \
                RETURN count(*) AS COUNT'
    result = tx.run(query)
    return result.data()

In [74]:
def delete_with_call_apoc(tx):

    query = "CALL apoc.periodic.commit( \
            'MATCH (n) WITH n LIMIT $limit DETACH DELETE n RETURN count(*) AS COUNT',\
            {limit: 10000}\
            )\
            YIELD updates, executions, runtime, batches\
            RETURN updates, executions, runtime, batches;"
    
    result = tx.run(query)
    return result.data()

In [89]:
def create_node_1(tx, props_list):
    query = "   UNWIND $props_list AS map \
                MERGE (n:NATIONAL_ROAD_POINT {x:map.x_1, y:map.y_1}) \
                RETURN count(n) AS COUNT"
    
    result = tx.run(query, props_list = props_list)
    return result.data()

In [88]:
def create_node_2(tx, props_list):
    query = "   UNWIND $props_list AS map \
                MERGE (n:NATIONAL_ROAD_POINT {x:map.x_2, y:map.y_2}) \
                RETURN count(n) AS COUNT"
    
    result = tx.run(query, props_list = props_list)
    return result.data()

In [94]:
def create_relationships(tx, props_list):
    query = "   UNWIND $props_list AS map \
                MATCH (n_1:NATIONAL_ROAD_POINT {x:map.x_1, y:map.y_1}) \
                MATCH (n_2:NATIONAL_ROAD_POINT {x:map.x_2, y:map.y_2}) \
                MERGE (n_1)-[r:LINKED_TO]->(n_2) \
                SET r = map \
                RETURN count(r) AS COUNT"
    
    result = tx.run(query, props_list = props_list)
    return result.data()

In [76]:
def match_alone_nodes(tx):
    query = " MATCH (r) \
            WHERE NOT (r)-[:LINKED_TO]-() \
            RETURN r.road_index AS road_index"
    
    result = tx.run(query)
    return result.data()

Drivers session for Neo4j :

In [77]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", "passwordneo4j"))

Ardoise (pour effacer tout ancien graph)

In [90]:
with driver.session() as session:

    count_list = []

    #result = session.execute_write(delete_all)
    result = session.execute_write(delete_with_call_apoc)
driver.close()

#nb_nodes = result[0]['COUNT']
                
print(f'Total Numbers of deleted nodes: {result}')

Total Numbers of deleted nodes: [{'updates': 0, 'executions': 0, 'runtime': 0, 'batches': 1}]


In [87]:
props_list = tmja_wrs84_reduced.to_dict('records')

In [91]:
with driver.session() as session:

    result = session.execute_write(create_node_1, props_list)

driver.close()

nb_rel = result[0]['COUNT']

print(f'Total Numbers of relationships created/updates: {nb_rel}')  

Total Numbers of relationships created/updates: 5968


In [92]:
with driver.session() as session:

    result = session.execute_write(create_node_2, props_list)

driver.close()

nb_rel = result[0]['COUNT']

print(f'Total Numbers of relationships created/updates: {nb_rel}')  

Total Numbers of relationships created/updates: 5968


In [95]:
with driver.session() as session:

    result = session.execute_write(create_relationships, props_list)

driver.close()

nb_rel = result[0]['COUNT']

print(f'Total Numbers of relationships created/updates: {nb_rel}')  



Total Numbers of relationships created/updates: 5968


On regarde le nombres de noeuds orphelins :

In [96]:
with driver.session() as session:

    result = session.execute_write(match_alone_nodes)

driver.close()

print(f'Total Numbers of alone nodes: {len(result)}')  

Total Numbers of alone nodes: 0


In [312]:
alone_nodes = pd.DataFrame(result)['road_index'].to_list()

On réitère le processus pour les noeuds orphelins, en trouvant un stratagème pour éviter la multiplication des neouds...

Définition d'un buffer -size adéquat en geopandas, pour agrandir les points routes

In [313]:
df_for_alone_nodes = prepare_df_for_graph_with_buffer(tmja_wrs84_clean, buffer_size=0.1)

/tmp/ipykernel_291858/3472978905.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_buf = df.buffer(buffer_size)
/home/jesshuan/miniconda3/envs/ECartoMobile/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


On ne garde que les lignes du dataframe où se trouvent des noeuds orphelins, et on ordonne le tout par distance, pour prioritiser les relations à construire sur ces neouds :

In [315]:
to_keep = (df_for_alone_nodes['road_index_left'].isin(alone_nodes)) | (df_for_alone_nodes['road_index_right'].isin(alone_nodes))

df_alone_buffer = df_for_alone_nodes.loc[to_keep, :].sort_values('distance')

In [316]:
df_alone_buffer

,road_index_left,road_index_right,Road_center_left,Road_center_right,distance
5544,5544,5543,POINT (3.13463 50.62348),POINT (3.13496 50.62338),38.338070
5543,5543,5544,POINT (3.13496 50.62338),POINT (3.13463 50.62348),38.338070
5543,5543,5541,POINT (3.13496 50.62338),POINT (3.13559 50.62162),207.033320
5541,5541,5543,POINT (3.13559 50.62162),POINT (3.13496 50.62338),207.033320
5544,5544,5541,POINT (3.13463 50.62348),POINT (3.13559 50.62162),232.140783
...,...,...,...,...,...
801,801,5544,POINT (3.35149 50.43639),POINT (3.13463 50.62348),31740.017987
5412,5412,4993,POINT (-2.20416 48.42859),POINT (-2.00816 48.63950),31940.612682
4993,4993,5412,POINT (-2.00816 48.63950),POINT (-2.20416 48.42859),31940.612682
256,256,2571,POINT (4.88466 46.47649),POINT (4.84625 46.77432),33306.582504


On lance un algo qui permet de détecter un noeud orphelin dans une relation (à droite ou à gauche) une est une seule fois, grâce à une collection peu à peu construite :

In [317]:
road_index_left_list = df_alone_buffer['road_index_left'].tolist()
road_index_right_list = df_alone_buffer['road_index_right'].tolist()
length_df_alone = len(df_alone_buffer)
alone_index_viewed = set()

result_control_list = []

for index_left, index_right in zip(road_index_left_list, road_index_right_list):
    if index_left in alone_nodes and index_left not in alone_index_viewed \
        and index_right not in alone_nodes :
        result_control_list.append(True)
        alone_index_viewed.add(index_left)
    elif index_right in alone_nodes and index_right not in alone_index_viewed \
        and index_left not in alone_nodes :
        result_control_list.append(True)
        alone_index_viewed.add(index_right)
    else:
        result_control_list.append(False)

df_alone_buffer['control_view'] = result_control_list

Réduction du dataframe aux relations à construire, qui impliquent une et une seule fois chaque neoud orphelin :

In [319]:
df_alone_buffer = df_alone_buffer.loc[df_alone_buffer['control_view']==True,:]
df_alone_buffer.drop(['control_view'], axis=1, inplace=True)
df_alone_buffer

/tmp/ipykernel_291858/3334822826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alone_buffer.drop(['control_view'], axis=1, inplace=True)


,road_index_left,road_index_right,Road_center_left,Road_center_right,distance
5543,5543,5541,POINT (3.13496 50.62338),POINT (3.13559 50.62162),207.033320
5544,5544,5541,POINT (3.13463 50.62348),POINT (3.13559 50.62162),232.140783
2849,2849,2850,POINT (4.88275 44.90442),POINT (4.88081 44.90265),290.405376
307,307,4914,POINT (4.91972 43.89277),POINT (4.91556 43.89013),544.723871
257,257,4094,POINT (4.83567 46.36849),POINT (4.84104 46.36406),771.577616
2743,2743,2742,POINT (4.67558 45.82287),POINT (4.66790 45.82536),893.751133
4517,4517,1736,POINT (3.13271 45.77697),POINT (3.14087 45.77790),908.035667
1271,1271,1328,POINT (6.09493 46.13262),POINT (6.09793 46.14129),1016.321305
5949,5949,4440,POINT (3.48388 44.52552),POINT (3.48178 44.51534),1154.640031
2207,2207,1485,POINT (5.68430 45.10724),POINT (5.67525 45.10086),1225.649423


Préparation du dataframe poru le graph sous Neo4j :

In [321]:
props_list = df_alone_buffer[['road_index_left','road_index_right','distance']].to_dict('records')

In [322]:
with driver.session() as session:

    result = session.execute_write(create_relationships, props_list)

driver.close()

nb_rel = result[0]['COUNT']

print(f'Total Numbers of relationships created/updates: {nb_rel}')  

Total Numbers of relationships created/updates: 25


Vérification des neouds orphelins dans le graph :

In [15]:
with driver.session() as session:

    result = session.execute_write(match_alone_nodes)

driver.close()

print(f'Total Numbers of alone nodes: {len(result)}')  

Total Numbers of alone nodes: 0
